<a href="https://colab.research.google.com/github/Yashsshah909/ML-for-Cyber-Analytics/blob/main/CyberAnalyticsGColab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Import libraries

In [2]:
import pandas as pd
import numpy as np
import torch as nn

Mount google drive

In [3]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


Fetch the csv files. 

In [4]:
d1=pd.read_csv('/content/drive/MyDrive/Cyber Analytics/cic ids 2017/TrafficLabelling/Friday-WorkingHours-Afternoon-DDos.pcap_ISCX.csv');
d2=pd.read_csv('/content/drive/MyDrive/Cyber Analytics/cic ids 2017/TrafficLabelling/Friday-WorkingHours-Afternoon-PortScan.pcap_ISCX.csv');
d3=pd.read_csv('/content/drive/MyDrive/Cyber Analytics/cic ids 2017/TrafficLabelling/Friday-WorkingHours-Morning.pcap_ISCX.csv');
d4=pd.read_csv('/content/drive/MyDrive/Cyber Analytics/cic ids 2017/TrafficLabelling/Monday-WorkingHours.pcap_ISCX.csv');
d5=pd.read_csv('/content/drive/MyDrive/Cyber Analytics/cic ids 2017/TrafficLabelling/Thursday-WorkingHours-Afternoon-Infilteration.pcap_ISCX.csv');
d6=pd.read_csv('/content/drive/MyDrive/Cyber Analytics/cic ids 2017/TrafficLabelling/Thursday-WorkingHours-Morning-WebAttacks.pcap_ISCX.csv', nrows=170366,encoding = 'unicode_escape');
d7=pd.read_csv('/content/drive/MyDrive/Cyber Analytics/cic ids 2017/TrafficLabelling/Tuesday-WorkingHours.pcap_ISCX.csv');
d8=pd.read_csv('/content/drive/MyDrive/Cyber Analytics/cic ids 2017/TrafficLabelling/Wednesday-workingHours.pcap_ISCX.csv');


Data Cleaning

In [5]:
d6[d6['Flow ID'].isnull()].index.tolist()

[]

Combine all the csv files

In [6]:
frames = [d1,d2,d3,d4,d5,d6,d7,d8]
d = pd.concat(frames)

In [7]:
d.shape

(2830743, 85)

In [8]:
print(d.info())


<class 'pandas.core.frame.DataFrame'>
Int64Index: 2830743 entries, 0 to 692702
Data columns (total 85 columns):
 #   Column                        Dtype  
---  ------                        -----  
 0   Flow ID                       object 
 1    Source IP                    object 
 2    Source Port                  int64  
 3    Destination IP               object 
 4    Destination Port             int64  
 5    Protocol                     int64  
 6    Timestamp                    object 
 7    Flow Duration                int64  
 8    Total Fwd Packets            int64  
 9    Total Backward Packets       int64  
 10  Total Length of Fwd Packets   float64
 11   Total Length of Bwd Packets  float64
 12   Fwd Packet Length Max        float64
 13   Fwd Packet Length Min        float64
 14   Fwd Packet Length Mean       float64
 15   Fwd Packet Length Std        float64
 16  Bwd Packet Length Max         float64
 17   Bwd Packet Length Min        float64
 18   Bwd Packet Length Mean

Find null values in Flow Bytes

In [9]:
d['Flow Bytes/s'].isnull().sum() 

1358

Substitute null values with the mean of the values in the column

In [10]:
column_means = d['Flow Bytes/s'].mean()
d['Flow Bytes/s'] = d['Flow Bytes/s'].fillna(column_means)

In [11]:
d['Flow Bytes/s'].isnull().sum() 

0

Check for infinite values

In [12]:
c = np.isinf(d['Flow Bytes/s']).values.sum()
c

2867

Substitute inf values with the max value of the column

In [13]:
m = d.loc[d['Flow Bytes/s'] != np.inf, 'Flow Bytes/s'].max()
d['Flow Bytes/s'].replace(np.inf,m,inplace=True)

In [14]:
c=np.isinf(d['Flow Bytes/s']).values.sum()
c

0

Follow the same procedure for column Flow Packets

In [15]:
d[' Flow Packets/s'].isnull().sum() 

0

In [16]:
e = np.isinf(d[' Flow Packets/s']).values.sum()
e

2867

In [17]:
m = d.loc[d[' Flow Packets/s'] != np.inf, ' Flow Packets/s'].max()
d[' Flow Packets/s'].replace(np.inf,m,inplace=True)

In [18]:
e = np.isinf(d[' Flow Packets/s']).values.sum()
e

0

In [19]:
d.isnull().sum().sum()

0

In [21]:
d

,Flow ID,Source IP,Source Port,Destination IP,Destination Port,Protocol,Timestamp,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Min,Bwd Packet Length Mean,Bwd Packet Length Std,Flow Bytes/s,Flow Packets/s,Flow IAT Mean,Flow IAT Std,Flow IAT Max,Flow IAT Min,Fwd IAT Total,Fwd IAT Mean,Fwd IAT Std,Fwd IAT Max,Fwd IAT Min,Bwd IAT Total,Bwd IAT Mean,Bwd IAT Std,Bwd IAT Max,Bwd IAT Min,Fwd PSH Flags,Bwd PSH Flags,Fwd URG Flags,Bwd URG Flags,...,Max Packet Length,Packet Length Mean,Packet Length Std,Packet Length Variance,FIN Flag Count,SYN Flag Count,RST Flag Count,PSH Flag Count,ACK Flag Count,URG Flag Count,CWE Flag Count,ECE Flag Count,Down/Up Ratio,Average Packet Size,Avg Fwd Segment Size,Avg Bwd Segment Size,Fwd Header Length.1,Fwd Avg Bytes/Bulk,Fwd Avg Packets/Bulk,Fwd Avg Bulk Rate,Bwd Avg Bytes/Bulk,Bwd Avg Packets/Bulk,Bwd Avg Bulk Rate,Subflow Fwd Packets,Subflow Fwd Bytes,Subflow Bwd Packets,Subflow Bwd Bytes,Init_Win_bytes_forward,Init_Win_bytes_backward,act_data_pkt_fwd,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
0,192.168.10.5-104.16.207.165-54865-443-6,104.16.207.165,443,192.168.10.5,54865,6,7/7/2017 3:30,3,2,0,12.0,0.0,6.0,6.0,6.0,0.00000,0.0,0.0,0.0,0.0,4.000000e+06,666666.666700,3.0,0.000000,3.0,3.0,3.0,3.00000,0.00000,3.0,3.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,...,6.0,6.000000,0.000000,0.000000,0,0,0,0,1,0,0,0,0.0,9.000000,6.0,0.0,40,0,0,0,0,0,0,2,12,0,0,33,-1,1,20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,BENIGN
1,192.168.10.5-104.16.28.216-55054-80-6,104.16.28.216,80,192.168.10.5,55054,6,7/7/2017 3:30,109,1,1,6.0,6.0,6.0,6.0,6.0,0.00000,6.0,6.0,6.0,0.0,1.100917e+05,18348.623850,109.0,0.000000,109.0,109.0,0.0,0.00000,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,...,6.0,6.000000,0.000000,0.000000,0,0,0,0,1,1,0,0,1.0,9.000000,6.0,6.0,20,0,0,0,0,0,0,1,6,1,6,29,256,0,20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,BENIGN
2,192.168.10.5-104.16.28.216-55055-80-6,104.16.28.216,80,192.168.10.5,55055,6,7/7/2017 3:30,52,1,1,6.0,6.0,6.0,6.0,6.0,0.00000,6.0,6.0,6.0,0.0,2.307692e+05,38461.538460,52.0,0.000000,52.0,52.0,0.0,0.00000,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,...,6.0,6.000000,0.000000,0.000000,0,0,0,0,1,1,0,0,1.0,9.000000,6.0,6.0,20,0,0,0,0,0,0,1,6,1,6,29,256,0,20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,BENIGN
3,192.168.10.16-104.17.241.25-46236-443-6,104.17.241.25,443,192.168.10.16,46236,6,7/7/2017 3:30,34,1,1,6.0,6.0,6.0,6.0,6.0,0.00000,6.0,6.0,6.0,0.0,3.529412e+05,58823.529410,34.0,0.000000,34.0,34.0,0.0,0.00000,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,...,6.0,6.000000,0.000000,0.000000,0,0,0,0,1,1,0,0,1.0,9.000000,6.0,6.0,20,0,0,0,0,0,0,1,6,1,6,31,329,0,20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,BENIGN
4,192.168.10.5-104.19.196.102-54863-443-6,104.19.196.102,443,192.168.10.5,54863,6,7/7/2017 3:30,3,2,0,12.0,0.0,6.0,6.0,6.0,0.00000,0.0,0.0,0.0,0.0,4.000000e+06,666666.666700,3.0,0.000000,3.0,3.0,3.0,3.00000,0.00000,3.0,3.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,...,6.0,6.000000,0.000000,0.000000,0,0,0,0,1,0,0,0,0.0,9.000000,6.0,0.0,40,0,0,0,0,0,0,2,12,0,0,32,-1,1,20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,BENIGN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
692698,192.168.10.3-192.168.10.14-53-51114-17,192.168.10.14,51114,192.168.10.3,53,17,5/7/2017 12:10,32215,4,2,112.0,152.0,28.0,28.0,28.0,0.00000,76.0,76.0,76.0,0.0,8.194940e+03,186.248642,6443.0,13617.579480,30780.0,3.0,30832.0,10277.33333,17755.84381,30780.0,4.0,3.0,3.0,0.0,3.0,3.0,0,0,0,0,...,76.0,41.714286,23.421602,548.571429,0,0,0,0,0,0,0,0,0.0,48.666667,28.0,76.0,80,0,0,0,0,0,0,4

Dropping the columns which are insignificant. The values in these columns are all zero or a constant value throught the column

In [27]:
d = d.drop(columns = [' Bwd PSH Flags', ' Fwd URG Flags', ' Bwd URG Flags', ' CWE Flag Count', 'Fwd Avg Bytes/Bulk', ' Fwd Avg Packets/Bulk', ' Fwd Avg Bulk Rate', ' Bwd Avg Bytes/Bulk', ' Bwd Avg Packets/Bulk', 'Bwd Avg Bulk Rate'])

In [28]:
d

,Flow ID,Source IP,Source Port,Destination IP,Destination Port,Protocol,Timestamp,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Min,Bwd Packet Length Mean,Bwd Packet Length Std,Flow Bytes/s,Flow Packets/s,Flow IAT Mean,Flow IAT Std,Flow IAT Max,Flow IAT Min,Fwd IAT Total,Fwd IAT Mean,Fwd IAT Std,Fwd IAT Max,Fwd IAT Min,Bwd IAT Total,Bwd IAT Mean,Bwd IAT Std,Bwd IAT Max,Bwd IAT Min,Fwd PSH Flags,Fwd Header Length,Bwd Header Length,Fwd Packets/s,Bwd Packets/s,Min Packet Length,Max Packet Length,Packet Length Mean,Packet Length Std,Packet Length Variance,FIN Flag Count,SYN Flag Count,RST Flag Count,PSH Flag Count,ACK Flag Count,URG Flag Count,ECE Flag Count,Down/Up Ratio,Average Packet Size,Avg Fwd Segment Size,Avg Bwd Segment Size,Fwd Header Length.1,Subflow Fwd Packets,Subflow Fwd Bytes,Subflow Bwd Packets,Subflow Bwd Bytes,Init_Win_bytes_forward,Init_Win_bytes_backward,act_data_pkt_fwd,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
0,192.168.10.5-104.16.207.165-54865-443-6,104.16.207.165,443,192.168.10.5,54865,6,7/7/2017 3:30,3,2,0,12.0,0.0,6.0,6.0,6.0,0.00000,0.0,0.0,0.0,0.0,4.000000e+06,666666.666700,3.0,0.000000,3.0,3.0,3.0,3.00000,0.00000,3.0,3.0,0.0,0.0,0.0,0.0,0.0,0,40,0,666666.666700,0.000000,6.0,6.0,6.000000,0.000000,0.000000,0,0,0,0,1,0,0,0.0,9.000000,6.0,0.0,40,2,12,0,0,33,-1,1,20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,BENIGN
1,192.168.10.5-104.16.28.216-55054-80-6,104.16.28.216,80,192.168.10.5,55054,6,7/7/2017 3:30,109,1,1,6.0,6.0,6.0,6.0,6.0,0.00000,6.0,6.0,6.0,0.0,1.100917e+05,18348.623850,109.0,0.000000,109.0,109.0,0.0,0.00000,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,20,20,9174.311927,9174.311927,6.0,6.0,6.000000,0.000000,0.000000,0,0,0,0,1,1,0,1.0,9.000000,6.0,6.0,20,1,6,1,6,29,256,0,20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,BENIGN
2,192.168.10.5-104.16.28.216-55055-80-6,104.16.28.216,80,192.168.10.5,55055,6,7/7/2017 3:30,52,1,1,6.0,6.0,6.0,6.0,6.0,0.00000,6.0,6.0,6.0,0.0,2.307692e+05,38461.538460,52.0,0.000000,52.0,52.0,0.0,0.00000,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,20,20,19230.769230,19230.769230,6.0,6.0,6.000000,0.000000,0.000000,0,0,0,0,1,1,0,1.0,9.000000,6.0,6.0,20,1,6,1,6,29,256,0,20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,BENIGN
3,192.168.10.16-104.17.241.25-46236-443-6,104.17.241.25,443,192.168.10.16,46236,6,7/7/2017 3:30,34,1,1,6.0,6.0,6.0,6.0,6.0,0.00000,6.0,6.0,6.0,0.0,3.529412e+05,58823.529410,34.0,0.000000,34.0,34.0,0.0,0.00000,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,20,20,29411.764710,29411.764710,6.0,6.0,6.000000,0.000000,0.000000,0,0,0,0,1,1,0,1.0,9.000000,6.0,6.0,20,1,6,1,6,31,329,0,20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,BENIGN
4,192.168.10.5-104.19.196.102-54863-443-6,104.19.196.102,443,192.168.10.5,54863,6,7/7/2017 3:30,3,2,0,12.0,0.0,6.0,6.0,6.0,0.00000,0.0,0.0,0.0,0.0,4.000000e+06,666666.666700,3.0,0.000000,3.0,3.0,3.0,3.00000,0.00000,3.0,3.0,0.0,0.0,0.0,0.0,0.0,0,40,0,666666.666700,0.000000,6.0,6.0,6.000000,0.000000,0.000000,0,0,0,0,1,0,0,0.0,9.000000,6.0,0.0,40,2,12,0,0,32,-1,1,20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,BENIGN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
692698,192.168.10.3-192.168.10.14-53-51114-17,192.168.10.14,51114,192.168.10.3,53,17,5/7/2017 12:10,32215,4,2,112.0,152.0,28.0,28.0,28.0,0.00000,76.0,76.0,76.0,0.0,8.194940e+03,186.248642,6443.0,13617.579480,30780.0,3.0,30832.0,10277.33333,17755.84381,30780.0,4.0,3.0,3.0,0.0,3.0,3.0,0,80,64,124.165761,62.082881,28.0,76.0,41.714286,23.421602,548.571429,0,0,0,0,0,0,0,0.0,48.666667,28.0,76.0,80,4,112,2,152,-1,-1,3,20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,BENIGN
